In [1]:
import pandas as pd
import numpy as np
import xlrd
import sklearn
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import math
import datetime
import scipy.stats
import itertools
import pyodbc 
import oracledb 
import openpyxl
import cx_Oracle
import ast

# CAMP corporate VPN
## To extract the EHM customers

In [2]:
conn = oracledb.connect('#####')

selected_profile_IDs = pd.read_sql(f"""
SELECT * FROM V_ANALYZER_AC_DASHBOARD 
""", conn)


C:\Users\bezyb\AppData\Local\Temp\ipykernel_25680\51340323.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  selected_profile_IDs = pd.read_sql(f"""


In [3]:
data_5 = selected_profile_IDs.reset_index(drop=True)

#list_REGNO = pd.DataFrame(data_5['REGNO'].unique())
list_REGNO = (data_5['REGNO'].unique())
#list_REGNO.to_csv('list_REGNO_EHM.csv', index=0)
list_REGNO

array(['N373EF', 'I-FDIA', 'N506BA', ..., 'N915NX', 'N542WC', 'N548FX'],
      dtype=object)

In [4]:
data_5.columns

Index(['BILLABLESYSTEMOFRECORDID', 'ECTM_SUBSCRIPTION', 'DATASYSTEMOFRECORDID',
       'DAC', 'CUSTOMER', 'EHM_PRIMARYENTERPRISEID', 'ACSCORE',
       'ANALYZERACREVIEW', 'ACASSETID', 'ACMODEL', 'REGNO', 'ENGPOS',
       'ENGINESCORE', 'ANALYZERENGREVIEW', 'ENGASSETID', 'ESN', 'ENGOEM',
       'ENGMODEL', 'BASELINEDATE', 'LASTDATADATE', 'FILELASTDATADATE',
       'MANLASTDATADATE', 'FIRSTUPLOADAFTERLASTREVIEWDATE',
       'FIRSTUPLOADAFTERACTIVATIONDATE', 'ACTIVATIONDATE', 'LASTUPLOADDATE',
       'LASTREVIEWDATE', 'REVIEWAGE', 'INNEEDOFREVIEWAGE',
       'INNEEDOFREVIEWAGETYPE', 'OPENTRENDREQUESTCNT', 'CRITICALPERFORMANCE',
       'PERFORMANCEADVISORY', 'DATAQUALITY', 'SUSPECTMTX', 'INDICATIONISSUE',
       'INTERNAL', 'KNEEPADREVIEW', 'LASTREVIEWSTATUS', 'ISESCALATED',
       'PENDINGOEMFEEDBACK', 'DEFICIENTDATA', 'CSRFOLLOWUP', 'RUNDT',
       'ARCHIVEDATE', 'ID', 'EHMANALYSTLOCATION', 'SERVICENAME', 'RECORDERS',
       'PLOTGRAPHSERVICENAME'],
      dtype='object')

# CAMP corporate VPN

In [5]:
conn = oracledb.connect('####')

In [6]:
selected_profile_IDs = pd.read_sql(f"""

select
    e.NAME as ENTERPRISE_NAME,
    rr.DISPLAY_NAME as REPORT_TYPE,
    rs.NAME as SUBSCRIPTION_NAME,
    mca.REGNO,
    c.FIRSTNAME,
    c.LASTNAME,
    c.EMAIL,
    c.CONTACT_ID,
    DECODE(ler.ROLE_ID, '1', 'OWNER', '2', 'OPERATOR', '???') as ROLE,
    a.ADDR1,
    a.CITY
from EDW.MAJOR_COMPONENT mca
join SUBSCRIPTION.RPT_SUB_ITEM rsi on rsi.ITEM_ID = mca.ASSET_ID
join SUBSCRIPTION.RPT_SUB rs on rs.RPT_SUB_ID = rsi.RPT_SUB_ID
join CSI.ENTERPRISE e on e.ENTERPRISE_ID = rs.ENTERPRISE_ID
join SUBSCRIPTION.RPT_REPORT rr on rr.RPT_REPORT_ID = rs.RPT_REPORT_ID --and rr.DISPLAY_NAME = 'Data Receipt'
join SUBSCRIPTION.RPT_SUB_CONTACT rsc on rsc.RPT_SUB_ID = rs.RPT_SUB_ID
join CSI.CONTACT c on c.CONTACT_ID =rsc.CONTACT_PROFILE_ID
join CSI.LINKENTERPRISEREL ler
on      ler.STATUS = '0'
    and ler.CONTACT_ID = c.CONTACT_ID
    and ler.PROFILE_ID = '0'
    and ler.ROLE_ID in ('1', '2')
join CSI.ADDRESS a on a.ADDRESS_ID = ler.ADDRESS_ID
where   1=1
    --and mca.REGNO = 'XA-ABS'

    and rsi.IS_DELETED = 0
    and rs.IS_DELETED = 0
    and rsc.IS_DELETED = 0

    --and c.FIRSTNAME = 'BRAD'

order by
    e.NAME,
    rr.DISPLAY_NAME,
    rs.NAME,
    mca.REGNO,
    c.FIRSTNAME

        
""", conn)


data_3 = selected_profile_IDs.reset_index(drop=True)
data_3

C:\Users\bezyb\AppData\Local\Temp\ipykernel_25680\493216996.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  selected_profile_IDs = pd.read_sql(f"""


,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,CONTACT_ID,ROLE,ADDR1,CITY
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,800809.DB1,OPERATOR,30 N GOULD ST; STE R,SHERIDAN
1,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,800809.DB1,OWNER,30 N GOULD ST; STE R,SHERIDAN
2,0X25EED LLC,EHM Status Report,Fleet Status Report,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,800809.DB1,OWNER,30 N GOULD ST; STE R,SHERIDAN
3,0X25EED LLC,EHM Status Report,Fleet Status Report,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,800809.DB1,OPERATOR,30 N GOULD ST; STE R,SHERIDAN
4,0X25EED LLC,Trend Alert,Trend Alert,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,800809.DB1,OWNER,30 N GOULD ST; STE R,SHERIDAN
...,...,...,...,...,...,...,...,...,...,...,...
440852,ZVIKA GRINBERG,Delinquent Data Alert,DDA,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,657944.DB1,OPERATOR,"1 ST FREEDLANDER P.O BOX581, HADERA, ISRAEL 38...",HADERA
440853,ZVIKA GRINBERG,EHM Status Report,EHM Status Report,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,657944.DB1,OPERATOR,"1 ST FREEDLANDER P.O BOX581, HADERA, ISRAEL 38...",HADERA
440854,ZVIKA GRINBERG,EHM Status Report,EHM Status Report,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,657944.DB1,OWNER,"1 ST FREEDLANDER P.O BOX581, HADERA, ISRAEL 38...",HADERA
440855,ZVIKA GRINBERG,Trend Alert,Trend Alerts,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,657944.DB1,OPERATOR,"1 ST FREEDLANDER P.O BOX581, HADERA, ISRAEL 38...",HADERA


In [7]:
contact_info = data_3

contact_info.to_csv('contact_info.csv', index=0)

In [8]:
contact_info = pd.read_csv('contact_info.csv')
#contact_info

In [9]:
contact_info_including_duplicate_frequency_EHM = ( contact_info[contact_info['REGNO'].isin(list_REGNO)] )
contact_info_including_duplicate_frequency_EHM = contact_info_including_duplicate_frequency_EHM [contact_info_including_duplicate_frequency_EHM ['ROLE']=='OPERATOR'].reset_index(drop=True)
contact_info_including_duplicate_frequency_EHM = contact_info_including_duplicate_frequency_EHM.drop(columns=['CONTACT_ID','ADDR1','CITY'])
contact_info_including_duplicate_frequency_EHM


,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
1,0X25EED LLC,EHM Status Report,Fleet Status Report,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
2,0X25EED LLC,Trend Alert,Trend Alert,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
3,"1, 4GROUP, INC",Delinquent Data Alert,Retail,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
4,"1, 4GROUP, INC",Delinquent Data Alert,Retail,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
...,...,...,...,...,...,...,...,...
227095,ZULULAND PUBLISHING CC,EHM Status Report,Fleet Status Report - 73088,ZS-CCW,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
227096,ZULULAND PUBLISHING CC,EHM Status Report,Fleet Status Report - 73088,ZS-FLJ,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
227097,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-BVO,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
227098,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-CCW,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR


In [10]:
contact_info_including_duplicate_frequency_EHM['EMAIL'].unique()

array(['JAKE@GAMMA.AERO', 'N999CB@14GROUP.COM', 'LANCE@TRUMBLE.CA', ...,
       'BOGUSLAW.WYPYCHEWICZ@GMAIL.COM', 'GILLIAM@COLLINSPROP.CO.ZA',
       'ZULUJETS@GMAIL.COM'], dtype=object)

In [11]:
contact_info_including_duplicate_frequency_EHM = contact_info_including_duplicate_frequency_EHM.drop_duplicates().reset_index(drop=True)
contact_info_including_duplicate_frequency_EHM
total_data_set = contact_info_including_duplicate_frequency_EHM
total_data_set

,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
1,0X25EED LLC,EHM Status Report,Fleet Status Report,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
2,0X25EED LLC,Trend Alert,Trend Alert,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
3,"1, 4GROUP, INC",Delinquent Data Alert,Retail,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
4,"1, 4GROUP, INC",EHM Status Report,Fleet Status Report,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
...,...,...,...,...,...,...,...,...
197184,ZULULAND PUBLISHING CC,EHM Status Report,Fleet Status Report - 73088,ZS-CCW,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
197185,ZULULAND PUBLISHING CC,EHM Status Report,Fleet Status Report - 73088,ZS-FLJ,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
197186,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-BVO,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
197187,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-CCW,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR


In [ ]:
data = contact_info_including_duplicate_frequency_EHM

list_emails =  list(data['EMAIL'].unique())

results = []

for i in list_emails:
    list_REGNO = list( data [data['EMAIL']== i] ['REGNO'].unique() )
    
    for j in list_REGNO:
        list_ENTERPRISE_NAME = list( data [ (data['EMAIL']==i) & (data['REGNO']== j) ] ['ENTERPRISE_NAME'].unique())
        
        for k in list_ENTERPRISE_NAME:
            list_REPORT_TYPE = list( data [ (data['EMAIL']==i) & (data['REGNO']== j) & (data['ENTERPRISE_NAME']== k) ] ['REPORT_TYPE'].unique())

            
            for b in list_REPORT_TYPE:


                
                # Filter the DataFrame based on the conditions
                p = data[(data['EMAIL'] == i) & (data['REGNO'] == j) & (data['ENTERPRISE_NAME'] == k) & (data['REPORT_TYPE'] == b)]

                # Create a new DataFrame for each combination
                w = pd.DataFrame(p.iloc[0, :-2]).T
                w['Duplicates'] = len(p)
                w['Duplicates List in SUBSCRIPTION_NAME'] = ', '.join(p['SUBSCRIPTION_NAME'])
                w.reset_index(drop=True, inplace=True)

                # Append the result to the list of results
                results.append(w)

# Concatenate all the DataFrames in the results list into a single DataFrame
final_result = pd.concat(results, ignore_index=True)
final_result

In [ ]:
import pandas as pd

data = contact_info_including_duplicate_frequency_EHM

# Group by unique combinations of EMAIL, REGNO, ENTERPRISE_NAME, and REPORT_TYPE
grouped = data.groupby(['EMAIL', 'REGNO', 'ENTERPRISE_NAME', 'REPORT_TYPE'], as_index=False)

# Initialize lists to store results
duplicates_count = []
duplicates_list = []

# Iterate over each group
for name, group in grouped:
    # Count duplicates
    duplicates_count.append(len(group) - 1)  # Subtract 1 to exclude the original record
    # Concatenate SUBSCRIPTION_NAME for duplicates
    
    subscription_names = group['SUBSCRIPTION_NAME'].unique().tolist()
    # Append the list to the duplicates_list
    duplicates_list.append(subscription_names)



# Create a DataFrame to hold the results
final_result = grouped.first()  # Get the first row of each group
final_result['Duplicates'] = duplicates_count
final_result['Duplicates List in SUBSCRIPTION_NAME'] = duplicates_list


In [13]:
final_result

,EMAIL,REGNO,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,FIRSTNAME,LASTNAME,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME
0,ALPHAREVI@GMAIL.COM,A-0801,TNI-AU INDONESIAN AIR FORCE,Delinquent Data Alert,ESP,MUHAMMAD,RAHMATULLOH,OPERATOR,0,[ESP]
1,ALPHAREVI@GMAIL.COM,A-0801,TNI-AU INDONESIAN AIR FORCE,Fault Code Alert,Fault Code Alert,MUHAMMAD,RAHMATULLOH,OPERATOR,0,[Fault Code Alert]
2,ALPHAREVI@GMAIL.COM,A-0808,TNI-AU INDONESIAN AIR FORCE,Delinquent Data Alert,Delinquent Data Alert,MUHAMMAD,RAHMATULLOH,OPERATOR,0,[Delinquent Data Alert]
3,CAM@VALORK.COM.AU,VH-KXM,VALOR ‘K PTY LTD,Delinquent Data Alert,PA,GRAHAME,MURRAY,OPERATOR,0,[PA]
4,CAM@VALORK.COM.AU,VH-KXM,VALOR ‘K PTY LTD,EHM Status Report,EHM Status Report,GRAHAME,MURRAY,OPERATOR,0,[EHM Status Report]
...,...,...,...,...,...,...,...,...,...,...
184477,ZYL199807@163.COM,B-9807,AVIC HEBEI GENERAL AVIATION CO. LTD,Fault Code Alert,Fault Code Alert,ZHANG,YULONG,OPERATOR,0,[Fault Code Alert]
184478,ZYL199807@163.COM,B-9807,AVIC HEBEI GENERAL AVIATION CO. LTD,Trend Alert,Trend Alert,ZHANG,YULONG,OPERATOR,0,[Trend Alert]
184479,mike@jetzaviation.com,C-FPBX,1443324 ALBERTA LTD.,Delinquent Data Alert,DDA-HONEYWELL3,MIKE,ZAPORZAN,OPERATOR,0,[DDA-HONEYWELL3]
184480,mike@jetzaviation.com,C-FPBX,SAULTS & POLLARD REAL ESTATE LTD.,EHM Status Report,Fleet Status Report - 76591,MIKE,ZAPORZAN,OPERATOR,0,[Fleet Status Report - 76591]


In [14]:
final_result = final_result.sort_values(by='ENTERPRISE_NAME')
final_result.to_csv('Duplicate_EMAILS_SUBSCRIPTION_NAME.csv', index=0)
final_result

,EMAIL,REGNO,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,FIRSTNAME,LASTNAME,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME
71105,JAKE@GAMMA.AERO,N715JK,0X25EED LLC,EHM Status Report,Fleet Status Report,JAKE,RILEY,OPERATOR,0,[Fleet Status Report]
71106,JAKE@GAMMA.AERO,N715JK,0X25EED LLC,Trend Alert,Trend Alert,JAKE,RILEY,OPERATOR,0,[Trend Alert]
71104,JAKE@GAMMA.AERO,N715JK,0X25EED LLC,Delinquent Data Alert,AVIALL,JAKE,RILEY,OPERATOR,0,[AVIALL]
126737,N999CB@14GROUP.COM,N999CB,"1, 4GROUP, INC",EHM Status Report,Fleet Status Report,BRYAN,CARSON,OPERATOR,0,[Fleet Status Report ]
126736,N999CB@14GROUP.COM,N999CB,"1, 4GROUP, INC",Delinquent Data Alert,Retail,BRYAN,CARSON,OPERATOR,0,[Retail]
...,...,...,...,...,...,...,...,...,...,...
94754,KEVINCAMERON53@GMAIL.COM,ZS-FLJ,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,KEVIN,CAMERON,OPERATOR,0,[Trend Alert - 84541]
94750,KEVINCAMERON53@GMAIL.COM,ZS-CCW,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,KEVIN,CAMERON,OPERATOR,0,[Trend Alert - 84541]
184463,ZVIKA@KISLEV.CO.IL,4X-BHZ,ZVIKA GRINBERG,Trend Alert,Trend Alerts,ZVIKA,GRINBERG,OPERATOR,0,[Trend Alerts]
184462,ZVIKA@KISLEV.CO.IL,4X-BHZ,ZVIKA GRINBERG,EHM Status Report,EHM Status Report,ZVIKA,GRINBERG,OPERATOR,0,[EHM Status Report]


In [15]:
Duplicate_EMAILS_SUBSCRIPTION_NAME = pd.read_csv('Duplicate_EMAILS_SUBSCRIPTION_NAME.csv')


In [16]:
Duplicate_More_Than_One = Duplicate_EMAILS_SUBSCRIPTION_NAME [ Duplicate_EMAILS_SUBSCRIPTION_NAME ['Duplicates']>=1]
Duplicate_More_Than_One = Duplicate_More_Than_One.reset_index(drop=True)
Duplicate_More_Than_One

,EMAIL,REGNO,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,FIRSTNAME,LASTNAME,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME
0,HAUGHTONS.CTR@GINL.STATE.GOV,N85516,AAR GOVERNMENT SERVICES,EHM Status Report,EHM Status Report N85516,STEPHEN,HAUGHTON,OPERATOR,1,"['EHM Status Report N85516', 'EHM all']"
1,FLEISCHMANJ@GINL.STATE.GOV,N800AW,AAR GOVERNMENT SERVICES,EHM Status Report,Fleet Status Report All,JAMIE,FLEISCHMAN,OPERATOR,1,"['Fleet Status Report All', 'Fleet Status Repo..."
2,ULLOAMA@STATE.GOV,N568AW,AAR GOVERNMENT SERVICES,Delinquent Data Alert,Delinquent Data Alerts,MIGUEL,ULLOA,OPERATOR,1,"['Delinquent Data Alerts', 'Retail-DHC-8']"
3,FLEISCHMANJ@GINL.STATE.GOV,N563AW,AAR GOVERNMENT SERVICES,EHM Status Report,Fleet Status Report All,JAMIE,FLEISCHMAN,OPERATOR,1,"['Fleet Status Report All', 'Fleet Status Repo..."
4,FLEISCHMANJ@GINL.STATE.GOV,N567WK,AAR GOVERNMENT SERVICES,EHM Status Report,Fleet Status Report All,JAMIE,FLEISCHMAN,OPERATOR,1,"['Fleet Status Report All', 'Fleet Status Repo..."
...,...,...,...,...,...,...,...,...,...,...
9798,OPS@SAFARI-AIR.COM,A2-KYS,XUGANA AIR ( PTY) T/A SAFARI AIR,Trend Alert,Trend Alerts,THUTO,BATSHIDI,OPERATOR,1,['Trend Alerts']
9799,S.NEPAL@YETIAIRLINES.COM,9N-AMM,YETI AIRLINES DOMESTIC PVT.LTD,Delinquent Data Alert,DDA FMP,SANTOSH,NEPAL,OPERATOR,1,"['DDA FMP', 'Retail']"
9800,M.ABYAT@YAHOO.COM,5H-TAQ,ZANTAS AIR SERVICES LIMITED,EHM Status Report,Fleet Status Report - 67571,MOHAMED,ABYAT,OPERATOR,1,"['Fleet Status Report - 67571', 'Fleet Status ..."
9801,M.ABYAT@YAHOO.COM,5H-FZA,ZANTAS AIR SERVICES LIMITED,EHM Status Report,Fleet Status Report - 67571,MOHAMED,ABYAT,OPERATOR,1,"['Fleet Status Report - 67571', 'Fleet Status ..."


In [19]:
list_ENTERPRISE_NAME = list ( Duplicate_More_Than_One['ENTERPRISE_NAME'].unique())
len(list_ENTERPRISE_NAME)

407

In [18]:
total = []


for i in list_ENTERPRISE_NAME[:]:
    #i = 'AAR GOVERNMENT SERVICES'
    selected_data = Duplicate_More_Than_One[Duplicate_More_Than_One['ENTERPRISE_NAME'] == i ].sort_values(by=['REPORT_TYPE', 'FIRSTNAME']).reset_index(drop=True)
    list_REPORT_TYPE = list(selected_data['REPORT_TYPE'].unique())

    
    for j in list_REPORT_TYPE:
    #j = 'Delinquent Data Alert'
        selected_data_REPORT_TYPE = selected_data[selected_data['REPORT_TYPE'] == j]
    
        list_SUBSCRIPTION_NAME = list ( selected_data_REPORT_TYPE ['Duplicates List in SUBSCRIPTION_NAME'].unique() )
    
    
        total_data_set_SELECTION = total_data_set[ (total_data_set['ENTERPRISE_NAME'] == i) & (total_data_set['REPORT_TYPE'] == j) ].sort_values(by=['REPORT_TYPE', 'FIRSTNAME']).reset_index(drop=True)
    
        unique_counts = total_data_set_SELECTION.groupby('SUBSCRIPTION_NAME')['REGNO'].apply(lambda x: len(set(x)))
        unique_counts = unique_counts.sort_values(ascending=False)
        High_to_Low_REGNO_Affected = list(unique_counts.index)
    
        df = pd.DataFrame(unique_counts)
        High_to_Low_REGNO_Affected = df.apply(lambda row: f"{row.name} ({row['REGNO']} Aircrafts) ", axis=1).tolist()


        
        REGNO_total_affected_on_SUBSCRIPTION_NAMES = pd.DataFrame(unique_counts).index.to_list()

        
        
        for k in list_SUBSCRIPTION_NAME:
            
            selected_data_List_SUBSCRIPTION_NAME = selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == k]
            df = selected_data_List_SUBSCRIPTION_NAME[['FIRSTNAME', 'LASTNAME']].drop_duplicates()
            full_names = ', '.join(df.apply(lambda x: f"{x['FIRSTNAME']} {x['LASTNAME']}", axis=1))
            EMAILS = (selected_data_List_SUBSCRIPTION_NAME['EMAIL']).unique()

            Affected_REGNO = list((selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == k])['REGNO'].unique())
            Affected_REGNO_quantities = len(Affected_REGNO)
        
            Duplicates = len(k.split(","))
        
            data_to_be_removed = [[i, j, full_names, EMAILS , Duplicates, f'{len(k.split(","))-1} ',Affected_REGNO, Affected_REGNO_quantities , k
                                   , High_to_Low_REGNO_Affected, REGNO_total_affected_on_SUBSCRIPTION_NAMES ]]
        
            data_final = pd.DataFrame(data_to_be_removed, columns=['ENTERPRISE_NAME', 'REPORT_TYPE', 'Full_Names', 'Customers EMAILS' , 'Duplicates', 'How many Emails should be removed?'
                                                                   ,'Affected_REGNO', 'Affected_REGNO_quantities', 'Duplicates_in_SUBSCRIPTION_NAMES'
                                                                   ,'High_to_Low_REGNO_Quantity_Affected', 'REGNO_total_affected_on_SUBSCRIPTION_NAMES' ])




            list1 = (data_final['REGNO_total_affected_on_SUBSCRIPTION_NAMES'][0])
            list2 = data_final['Duplicates_in_SUBSCRIPTION_NAMES']
            list2 = ast.literal_eval(list2[0])
            
            items = []
            for item in list1:
                if item in list2:
                    items.append(item)

            # Define a function to handle the condition
            def assign_values(row):
                if row['Duplicates'] > 1:
                    return pd.Series({'DONT Remove from': items[0], 'Remove from': items[1:]})
                else:
                    return pd.Series({'DONT Remove from': np.nan, 'Remove from': np.nan})
            
            # Apply the function to each row of the DataFrame
            data_final[['DONT Remove from', 'Remove from']] = data_final.apply(assign_values, axis=1)
        


            total.append(data_final)

# Check if the total list is not empty before concatenating the DataFrames
if total:
    final_result = pd.concat(total, ignore_index=True)
    final_result
else:
    print("No data to concatenate.")

final_result['Duplicates'] = final_result['Duplicates'].replace(1, 0)
#final_result['Affected_REGNO_quantities'] = final_result['Affected_REGNO'].apply(lambda x: len(x))
final_result.to_csv('Emails_to_be removed_from_customers_accounts.csv' , index=0)


final_result

,ENTERPRISE_NAME,REPORT_TYPE,Full_Names,Customers EMAILS,Duplicates,How many Emails should be removed?,Affected_REGNO,Affected_REGNO_quantities,Duplicates_in_SUBSCRIPTION_NAMES,High_to_Low_REGNO_Quantity_Affected,REGNO_total_affected_on_SUBSCRIPTION_NAMES,DONT Remove from,Remove from
0,AAR GOVERNMENT SERVICES,Data Receipt,MELISSA PAGE,[PAGEM.CTR@GINL.STATE.GOV],2,1,"[N570AW, N589AW, N568AW]",3,"['Data Receipt-Fleet', 'Data Receipts ']","[Data Receipt-Fleet (5 Aircrafts) , Data Recei...","[Data Receipt-Fleet, Data Receipts-DHC-8, Data...",Data Receipt-Fleet,[Data Receipts ]
1,AAR GOVERNMENT SERVICES,Data Receipt,MICHAEL CASEY,[CASEYM.CTR@GINL.STATE.GOV],2,1,[N568AW],1,"['Data Receipt N568AW', 'Data Receipts-DHC-8']","[Data Receipt-Fleet (5 Aircrafts) , Data Recei...","[Data Receipt-Fleet, Data Receipts-DHC-8, Data...",Data Receipts-DHC-8,[Data Receipt N568AW]
2,AAR GOVERNMENT SERVICES,Delinquent Data Alert,ADAM MESSING,[MESSINGA.CTR@GINL.STATE.GOV],2,1,[N568AW],1,"['DDA N568AW', 'Delinquent Data Alert']","[Retail-Fleet (5 Aircrafts) , Retail-DHC-8 (4 ...","[Retail-Fleet, Retail-DHC-8, DDA, Delinquent D...",Delinquent Data Alert,[DDA N568AW]
3,AAR GOVERNMENT SERVICES,Delinquent Data Alert,"ANDREW GEE, CHRISTOPHER ROBERTS, JEFFREY UNDE...","[GEEA@GINL.STATE.GOV, CHRISTOPHER.ROBERTS@AARC...",2,1,"[N568AW, N570AW, N589AW]",3,"['Delinquent Data Alerts', 'Retail-DHC-8']","[Retail-Fleet (5 Aircrafts) , Retail-DHC-8 (4 ...","[Retail-Fleet, Retail-DHC-8, DDA, Delinquent D...",Retail-DHC-8,[Delinquent Data Alerts]
4,AAR GOVERNMENT SERVICES,Delinquent Data Alert,BENJAMIN AYERS,[AYERSB.CTR@GINL.STATE.GOV],2,1,"[N589AW, N570AW]",2,"['DDA', 'Delinquent Data Alerts']","[Retail-Fleet (5 Aircrafts) , Retail-DHC-8 (4 ...","[Retail-Fleet, Retail-DHC-8, DDA, Delinquent D...",DDA,[Delinquent Data Alerts]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413,XUGANA AIR ( PTY) T/A SAFARI AIR,Fault Code Alert,THUTO BATSHIDI,[OPS@SAFARI-AIR.COM],0,0,[A2-KYS],1,['Fault Code Alert'],[Fault Code Alert (1 Aircrafts) ],[Fault Code Alert],NaN,NaN
1414,XUGANA AIR ( PTY) T/A SAFARI AIR,Trend Alert,THUTO BATSHIDI,[OPS@SAFARI-AIR.COM],0,0,[A2-KYS],1,['Trend Alerts'],[Trend Alerts (1 Aircrafts) ],[Trend Alerts],NaN,NaN
1415,YETI AIRLINES DOMESTIC PVT.LTD,Delinquent Data Alert,SANTOSH NEPAL,[S.NEPAL@YETIAIRLINES.COM],2,1,[9N-AMM],1,"['DDA FMP', 'Retail']","[DDA FMP (4 Aircrafts) , Retail (4 Aircrafts) ]","[DDA FMP, Retail]",DDA FMP,[Retail]
1416,ZANTAS AIR SERVICES LIMITED,EHM Status Report,MOHAMED ABYAT,[M.ABYAT@YAHOO.COM],2,1,"[5H-TAQ, 5H-FZA]",2,"['Fleet Status Report - 67571', 'Fleet Status ...","[Fleet Status Report - 67571 (3 Aircrafts) , F...","[Fleet Status Report - 67571, Fleet Status Rep...",Fleet Status Report - 67571,[Fleet Status Report - 71036]


In [85]:
regno_count

[3, 3, 0, 1, 5, 3, 0, 4, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [86]:
max_count_regno

1

In [81]:
w = pd.DataFrame([list_Duplicates_in_SUBSCRIPTION_NAMES, [len(SUBSCRIPTION_NAME_df_REGNO)]], index=['SUBSCRIPTION_NAME', 'no_REGNO']).T

index_of_max_value = w['no_REGNO'].idxmax()
MAX_REGNO = w.loc[index_of_max_value, 'SUBSCRIPTION_NAME']
MAX_REGNO

'Trend Alert'

In [61]:
contact_info_including_duplicate_frequency_EHM [ (contact_info_including_duplicate_frequency_EHM ['ENTERPRISE_NAME']==i) & (contact_info_including_duplicate_frequency_EHM ['REPORT_TYPE'] == j)
                                        & (contact_info_including_duplicate_frequency_EHM ['SUBSCRIPTION_NAME']==r) ]

,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR


In [52]:
contact_info_including_duplicate_frequency_EHM [ contact_info_including_duplicate_frequency_EHM ]

,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
1,0X25EED LLC,EHM Status Report,Fleet Status Report,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
2,0X25EED LLC,Trend Alert,Trend Alert,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR
3,"1, 4GROUP, INC",Delinquent Data Alert,Retail,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
4,"1, 4GROUP, INC",EHM Status Report,Fleet Status Report,N999CB,BRYAN,CARSON,N999CB@14GROUP.COM,OPERATOR
...,...,...,...,...,...,...,...,...
195053,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-CCW,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
195054,ZULULAND PUBLISHING CC,Trend Alert,Trend Alert - 84541,ZS-FLJ,KEVIN,CAMERON,KEVINCAMERON53@GMAIL.COM,OPERATOR
195055,ZVIKA GRINBERG,Delinquent Data Alert,DDA,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,OPERATOR
195056,ZVIKA GRINBERG,EHM Status Report,EHM Status Report,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,OPERATOR


In [51]:
contact_info_including_duplicate_frequency_EHM [ (contact_info_including_duplicate_frequency_EHM ['ENTERPRISE_NAME']==i) & (contact_info_including_duplicate_frequency_EHM ['REPORT_TYPE'] == j)
                        & (contact_info_including_duplicate_frequency_EHM ['SUBSCRIPTION_NAME']==r) ]

,ENTERPRISE_NAME,REPORT_TYPE,SUBSCRIPTION_NAME,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE
0,0X25EED LLC,Delinquent Data Alert,AVIALL,N715JK,JAKE,RILEY,JAKE@GAMMA.AERO,OPERATOR


In [34]:
list ( data_final ['Duplicates_in_SUBSCRIPTION_NAMES'].unique() )

['Trend Alert']

In [ ]:
Duplicate_More_Than_One []

In [31]:
total = []

for i in list_ENTERPRISE_NAME[400:401]:
    selected_data = Duplicate_More_Than_One[Duplicate_More_Than_One['ENTERPRISE_NAME'] == i].sort_values(by=['REPORT_TYPE', 'FIRSTNAME']).reset_index(drop=True)
    list_REPORT_TYPE = list(selected_data['REPORT_TYPE'].unique())
    
    for j in list_REPORT_TYPE:
        selected_data_REPORT_TYPE = selected_data[selected_data['REPORT_TYPE'] == j]
        list_Duplicates_List_SUBSCRIPTION_NAME = list(selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'].unique())

        for k in list_Duplicates_List_SUBSCRIPTION_NAME:
            selected_data_List_SUBSCRIPTION_NAME = selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == k]
            df = selected_data_List_SUBSCRIPTION_NAME[['FIRSTNAME', 'LASTNAME']].drop_duplicates()
            full_names = ', '.join(df.apply(lambda x: f"{x['FIRSTNAME']} {x['LASTNAME']}", axis=1))

            EMAILS = (selected_data_List_SUBSCRIPTION_NAME['EMAIL']).unique()

            
            Affected_REGNO = list((selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == k])['REGNO'].unique())

            

            Duplicates = len(k.split(","))

            data_to_be_removed = [[i, j, full_names, EMAILS , Duplicates, f'{len(k.split(","))-1} ',k, Affected_REGNO]]

            data_final = pd.DataFrame(data_to_be_removed, columns=['ENTERPRISE_NAME', 'REPORT_TYPE', 'Full_Names', 'Customers EMAILS' , 'Duplicates', 'How many Emails should be removed?','Duplicates_in_SUBSCRIPTION_NAMES', 'Affected_REGNO'])




            w = Duplicate_More_Than_One [ (Duplicate_More_Than_One['ENTERPRISE_NAME'] == i ) & (Duplicate_More_Than_One['REPORT_TYPE'] == j ) ]
            list_SUB = w['Duplicates List in SUBSCRIPTION_NAME'].unique()
            t = []
            aircraft_Numbers = []
            for k in list_SUB:
                r = list (w [w['Duplicates List in SUBSCRIPTION_NAME'] == k]['REGNO'].unique())
                aircraft_Numbers.append(len(r))
                t.append(r)
            p = pd.DataFrame([t,aircraft_Numbers], columns=list_SUB, index = ['Affected_REGNO','Number_of_Affected_REGNO']).T
            h = p
            h

            

            index_of_max_value = data_final['Affected_REGNO_quantities'].idxmax()
            MAX_REGNO = data_final.loc[index_of_max_value, 'Duplicates_in_SUBSCRIPTION_NAMES']

            data_final ['DONT_remove_Emails_from_Below_categories'] = str ([MAX_REGNO])



            # Convert strings to lists
            list1 = list(data_final['Duplicates_in_SUBSCRIPTION_NAMES'])
            list2 = list(data_final['Affected_REGNO_quantities']) 
            result_list = [item for item in list1 if item not in list2]
            result_string = str(result_list)
            data_final ['Remove_Emails_from_Below_categories'] = result_string

            
            

            
            total.append(data_final)
# Check if the total list is not empty before concatenating the DataFrames
if total:
    final_result = pd.concat(total, ignore_index=True)
    final_result
else:
    print("No data to concatenate.")

final_result['Duplicates'] = final_result['Duplicates'].replace(1, 0)
final_result['Affected_REGNO_quantities'] = final_result['Affected_REGNO'].apply(lambda x: len(x))
#final_result.to_csv('Emails_to_be removed_from_customers_accounts.csv' , index=0)
final_result

KeyError: 'Affected_REGNO_quantities'

In [162]:
#u = final_result[(final_result['ENTERPRISE_NAME'] == 'AIR ARCHIPELS') & (final_result['REPORT_TYPE'] == 'Delinquent Data Alert')]
u = data_final
u

,ENTERPRISE_NAME,REPORT_TYPE,Full_Names,Customers EMAILS,Duplicates,How many Emails should be removed?,Duplicates_in_SUBSCRIPTION_NAMES,Affected_REGNO,Affected_REGNO_quantities,DONT_remove_Emails_from_Below_categories,Remove_Emails_from_Below_categories
0,AIR ARCHIPELS,Trend Alert,PATRICK MARAIS,[PATRICK.MARAIS@AIRARCHIPELS.PF],2,1,"Trend Alert - 79034, Trend Alerts for ADAS Fleet",[F-OIQK],1,"['Trend Alert - 79034, Trend Alerts for ADAS F...","['Trend Alert - 79034, Trend Alerts for ADAS F..."


In [149]:
index_of_max_value = u['Affected_REGNO_quantities'].idxmax()
MAX_REGNO = u.loc[index_of_max_value, 'Duplicates_in_SUBSCRIPTION_NAMES']
MAX_REGNO

'DDA'

In [146]:
u['DONT_remove_Emails_from_Below_categories'] = MAX_REGNO
u

C:\Users\bezyb\AppData\Local\Temp\ipykernel_18868\2845338859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  u['DONT_remove_Emails_from_Below_categories'] = MAX_REGNO


,ENTERPRISE_NAME,REPORT_TYPE,Full_Names,Customers EMAILS,Duplicates,How many Emails should be removed?,Duplicates_in_SUBSCRIPTION_NAMES,Affected_REGNO,Affected_REGNO_quantities,Remove_Emails_from_Below_categories,DONT_remove_Emails_from_Below_categories
0,AIR ARCHIPELS,Delinquent Data Alert,JEAN-PHILIPPE LORCA,[JP.LORCA@AIRARCHIPELS.PF],0,0,Delinquent Data Alerts for ADAS Fleet,[F-OIQK],1,['Delinquent Data Alerts for ADAS Fleet'],DDA
1,AIR ARCHIPELS,Delinquent Data Alert,"PATRICK MARAIS, TEIKI OMONT","[PATRICK.MARAIS@AIRARCHIPELS.PF, TEIKI.OMONT@A...",0,0,DDA,"[F-OSMU, F-OIPF, F-OIQK]",3,['DDA'],DDA
2,AIR ARCHIPELS,Delinquent Data Alert,PATRICK MARAIS,[PATRICK.MARAIS@AIRARCHIPELS.PF],2,1,"DDA, Delinquent Data Alerts for ADAS Fleet",[F-OIQK],1,"['DDA, Delinquent Data Alerts for ADAS Fleet']",DDA


In [115]:
list(data_final['Duplicates_in_SUBSCRIPTION_NAMES'])

['ESP']

In [106]:
['Fleet Status Report', 'Fleet Status Report-DH8']- ['Fleet Status Report']


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [83]:
(selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == 'Data Receipt-Fleet'])

,ENTERPRISE_NAME,REPORT_TYPE,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME


In [31]:
(selected_data_REPORT_TYPE[selected_data_REPORT_TYPE['Duplicates List in SUBSCRIPTION_NAME'] == k])

,ENTERPRISE_NAME,REPORT_TYPE,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME
2,ZVIKA GRINBERG,Trend Alert,4X-BHZ,ZVIKA,GRINBERG,ZVIKA@KISLEV.CO.IL,OPERATOR,1,Trend Alerts


In [167]:
w = Duplicate_More_Than_One [ (Duplicate_More_Than_One['ENTERPRISE_NAME'] =='AAR GOVERNMENT SERVICES' ) & (Duplicate_More_Than_One['REPORT_TYPE'] =='Data Receipt' ) ]

list_SUB = w
list_SUB

,ENTERPRISE_NAME,REPORT_TYPE,REGNO,FIRSTNAME,LASTNAME,EMAIL,ROLE,Duplicates,Duplicates List in SUBSCRIPTION_NAME
1234,AAR GOVERNMENT SERVICES,Data Receipt,N589AW,WILLIAM,MCLEAN,MCLEANW.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt-Fleet
1244,AAR GOVERNMENT SERVICES,Data Receipt,N684SA,WILLIAM,MCLEAN,MCLEANW.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt-Fleet
1249,AAR GOVERNMENT SERVICES,Data Receipt,N568AW,JAVIER,BARBOZA,BARBOZAJ.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt
1251,AAR GOVERNMENT SERVICES,Data Receipt,N570AW,ROBERT,DELEWSKI,DELEWSKIR.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipts
1253,AAR GOVERNMENT SERVICES,Data Receipt,N568AW,ROBERT,DELEWSKI,DELEWSKIR.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipts
...,...,...,...,...,...,...,...,...,...
2249,AAR GOVERNMENT SERVICES,Data Receipt,N568AW,LUIS,GARCIA,GARCIAACOSTAL.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt
2252,AAR GOVERNMENT SERVICES,Data Receipt,N589AW,CAMILO,GALLEGO,GALLEGOC.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt
2256,AAR GOVERNMENT SERVICES,Data Receipt,N684SA,RODRIGO,ARBOLEDA,ARBOLEDAR.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt - Peru N684SA
2264,AAR GOVERNMENT SERVICES,Data Receipt,N85516,RODRIGO,ARBOLEDA,ARBOLEDAR.CTR@GINL.STATE.GOV,OPERATOR,1,Data Receipt Peru N85516


In [34]:
w [w['Duplicates List in SUBSCRIPTION_NAME'] == 'Data Receipt-Fleet']['REGNO'].unique()

array(['N589AW', 'N684SA', 'N570AW', 'N568AW', 'N560WK'], dtype=object)

In [127]:
t = []
aircraft_Numbers = []
for k in list_SUB:
    r = list (w [w['Duplicates List in SUBSCRIPTION_NAME'] == k]['REGNO'].unique())

    aircraft_Numbers.append(len(r))

    t.append(r)

In [128]:
p = pd.DataFrame([t,aircraft_Numbers], columns=list_SUB, index = ['Affected_REGNO','Number_of_Affected_REGNO']).T
#h = p.sort_values(by=['Number_of_Affected_REGNO'], ascending=False)
h = p
h

,Affected_REGNO,Number_of_Affected_REGNO
Data Receipt-Fleet,"[N589AW, N684SA, N570AW, N568AW, N560WK]",5
Data Receipt,"[N568AW, N589AW]",2
Data Receipts,"[N570AW, N568AW, N589AW]",3
Data Receipt Peru N85516,[N85516],1
Data Receipts-DHC-8,"[N589AW, N570AW, N568AW, N560WK]",4
Data Receipt - Iraq,"[N569AW, N567WK, N800AW]",3
Data Receipt - Peru N684SA,[N684SA],1
"Data Receipt N568AW, Data Receipts-DHC-8",[N568AW],1
Data Receipt N568AW,[N568AW],1
Data Reciept N568AW,[N568AW],1


In [129]:
h['Number_of_Affected_REGNO'].idxmax()

'Data Receipt-Fleet'

In [60]:
row_lengths = df.apply(lambda row: len(row), axis=1)

In [61]:
df.apply(lambda x: len(x))

Data Receipt-Fleet                          1
Data Receipt                                1
Data Receipts                               1
Data Receipt Peru N85516                    1
Data Receipts-DHC-8                         1
Data Receipt - Iraq                         1
Data Receipt - Peru N684SA                  1
Data Receipt N568AW, Data Receipts-DHC-8    1
Data Receipt N568AW                         1
Data Reciept N568AW                         1
Data Receipt-Fleet, Data Receipts           1
dtype: int64

In [62]:
p

,Data Receipt-Fleet,Data Receipt,Data Receipts,Data Receipt Peru N85516,Data Receipts-DHC-8,Data Receipt - Iraq,Data Receipt - Peru N684SA,"Data Receipt N568AW, Data Receipts-DHC-8",Data Receipt N568AW,Data Reciept N568AW,"Data Receipt-Fleet, Data Receipts"
0,"[N589AW, N684SA, N570AW, N568AW, N560WK]","[N568AW, N589AW]","[N570AW, N568AW, N589AW]",[N85516],"[N589AW, N570AW, N568AW, N560WK]","[N569AW, N567WK, N800AW]",[N684SA],[N568AW],[N568AW],[N568AW],"[N570AW, N568AW, N589AW]"


In [69]:
(p['Data Receipt-Fleet'])

0    [N589AW, N684SA, N570AW, N568AW, N560WK]
Name: Data Receipt-Fleet, dtype: object

In [59]:
selected_profile_IDs = pd.read_sql(f"""

select * from
(
select CUSTOMER,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,EMAIL, count(*) as CNT from
(

select distinct
  e.NAME  AS CUSTOMER,E.ENTERPRISE_ID,rsi.ITEM_ID AS ASSET_ID,
  rr.DISPLAY_NAME AS ALERT_TYPE,
  rs.NAME as ALERT_NAME,
  v.REGNO,
  c.FIRSTNAME,
  c.LASTNAME,
  c.EMAIL


from SUBSCRIPTION.RPT_REPORT rr
join SUBSCRIPTION.RPT_SUB rs on rs.RPT_REPORT_ID = rr.RPT_REPORT_ID
join CSI.ENTERPRISE e on e.ENTERPRISE_ID = rs.ENTERPRISE_ID
join SUBSCRIPTION.RPT_SUB_ITEM rsi on rsi.RPT_SUB_ID = rs.RPT_SUB_ID
join SUBSCRIPTION.RPT_SUB_CONTACT rsc on rsc.RPT_SUB_ID = rs.RPT_SUB_ID
join CSI.CONTACT c on c.CONTACT_ID = rsc.CONTACT_PROFILE_ID and c.STATUS = '0'
JOIN CSI.LINKENTERPRISEREL ler ON ler.STATUS = '0' AND ler.CONTACT_ID = c.CONTACT_ID AND ler.PROFILE_ID = '0' and role_id <> 0
join SUBSCRIPTION.RPT_SUB_PROP rsp on rsp.RPT_SUB_ID = rs.RPT_SUB_ID and   rsp.IS_DELETED = 0
join SUBSCRIPTION.RPT_TRACKING rt on rt.RPT_SUB_ID = rs.RPT_SUB_ID and rt.IS_DELETED = 0
join TREND.V_ANALYZER_AC_DASHBOARD v on v.ACASSETID = rsi.ITEM_ID and ECTM_SUBSCRIPTION = 'Y' AND ENGPOS = 1
    /*LEFT JOIN (

                select distinct
                        asset.asset_id,
                        asset.serial_number,
                        asset.registration_number AS REGNO,
                        max(rpt_report_item_prop.from_date) as FROM_DATE,
                        max(rpt_report_item_prop.to_date) as TO_DATE
                from    subscription.rpt_report_item_prop
                        inner join trend.asset on asset.asset_id=rpt_report_item_prop.item_id
                where 
                        rpt_report_item_prop.is_deleted=0 
                    and rpt_report_item_prop.item_type='ASSET' and asset.is_deleted=0
                    group by asset.asset_id, asset.serial_number, asset.registration_number
                order by asset.registration_number ) SN ON SN.REGNO =   v.REGNO AND TO_DATE >= SYSDATE*/
WHERE 1=1

AND ( e.name not like '%DEMO%' AND   e.name not like '%TEST%') 

  and rr.IS_DELETED = 0
  and rs.IS_DELETED = 0
  and rsi.IS_DELETED = 0
  and rsi.ITEM_TYPE = 'ASSET'
  and rsc.IS_DELETED = 0
    and c.email is not null
  and rs.CREATED_BY_USER not in ('EHM_AUTO_GEN')

order by
  rr.DISPLAY_NAME,
  v.REGNO,  c.FIRSTNAME,  c.LASTNAME

)
GROUP BY CUSTOMER,ALERT_TYPE,REGNO,EMAIL,ENTERPRISE_ID,ASSET_ID
ORDER BY 5 DESC
)l
where cnt >1
ORDER BY CUSTOMER,REGNO,EMAIL,ALERT_TYPE

      
""", conn)


data_3 = selected_profile_IDs.reset_index(drop=True)
data_3

C:\Users\bezyb\AppData\Local\Temp\ipykernel_30280\1557496106.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  selected_profile_IDs = pd.read_sql(f"""


,CUSTOMER,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,EMAIL,CNT
0,247 AVIATION LTD,538084.DB1,121741,Delinquent Data Alert,G-LUSO,BRIAN.SPURLE@247AVIATION.CO.UK,2
1,AAR GOVERNMENT SERVICES,468656.DB1,62788,EHM Status Report,N560WK,CARLSONCM@STATE.GOV,2
2,AAR GOVERNMENT SERVICES,468656.DB1,62788,Delinquent Data Alert,N560WK,COXW.CTR@GINL.STATE.GOV,2
3,AAR GOVERNMENT SERVICES,468656.DB1,62788,EHM Status Report,N560WK,COXW.CTR@GINL.STATE.GOV,3
4,AAR GOVERNMENT SERVICES,468656.DB1,62788,Delinquent Data Alert,N560WK,CREWSR.CTR@GINL.STATE.GOV,2
...,...,...,...,...,...,...,...
9374,XUGANA AIR ( PTY) T/A SAFARI AIR,489450.DB1,24099,Trend Alert,A2-KYS,OPS@SAFARI-AIR.COM,2
9375,YETI AIRLINES DOMESTIC PVT.LTD,457147.DB1,109766,Delinquent Data Alert,9N-AMM,S.NEPAL@YETIAIRLINES.COM,2
9376,ZANTAS AIR SERVICES LIMITED,234037.DB1,27711,EHM Status Report,5H-FZA,M.ABYAT@YAHOO.COM,2
9377,ZANTAS AIR SERVICES LIMITED,234037.DB1,27711,Trend Alert,5H-FZA,M.ABYAT@YAHOO.COM,2


In [60]:
#list_REGNO = list(data_3['REGNO'].unique())

list_CUSTOMER = list(data_3['CUSTOMER'].unique())

dupl = []
for i in list_CUSTOMER:
    list_Emails = len ( (( data_3[data_3['CUSTOMER']==i] )['EMAIL']).unique())

    dupl.append(list_Emails)

dup_alert = pd.DataFrame ((list_CUSTOMER,dupl),index=['CUSTOMER', 'Duplicates']).T
dup_alert

,CUSTOMER,Duplicates
0,247 AVIATION LTD,1
1,AAR GOVERNMENT SERVICES,20
2,ABELAG AVIATION,1
3,ABU DHABI AVIATION,7
4,ACE AVIATION SERVICES CORPORATION,1
...,...,...
365,WRIGHT AIR SERVICE INC.,1
366,XOJET AVIATION LLC,3
367,XUGANA AIR ( PTY) T/A SAFARI AIR,1
368,YETI AIRLINES DOMESTIC PVT.LTD,1


In [61]:
dup_alert['Duplicates'].sum()

1031

In [52]:
dup_alert [ dup_alert ['Duplicates']==13]

,REGNO,Duplicates
1767,VH-ZXA,13
1768,VH-ZXB,13
1769,VH-ZXC,13
1770,VH-ZXD,13


In [25]:
(data_3.groupby('REGNO').count() )[['ALERT_TYPE']]

,ALERT_TYPE
REGNO,
0001051,3
003013,3
003016,3
003020,3
003025,3
...,...
ZS-UMB,5
ZS-XCG,3
ZS-XCO,3


In [9]:
data_3['REGNO'].unique()

array(['G-LUSO', 'N560WK', 'N563AW', ..., '9N-AMM', '5H-FZA', '5H-TAQ'],
      dtype=object)

In [6]:
data_3['ALERT_TYPE'].unique()

array(['Delinquent Data Alert', 'EHM Status Report', 'Trend Alert',
       'Fault Code Alert'], dtype=object)

In [8]:
len (data_3['REGNO'].unique())

17079

In [5]:
Alert_types = data_3[['CUSTOMER', 'REGNO', 'ALERT_TYPE', 'EMAIL']]
Alert_types

,CUSTOMER,REGNO,ALERT_TYPE,EMAIL
0,0X25EED LLC,N715JK,Delinquent Data Alert,JAKE@GAMMA.AERO
1,0X25EED LLC,N715JK,EHM Status Report,JAKE@GAMMA.AERO
2,"1, 4GROUP, INC",N999CB,Delinquent Data Alert,N999CB@14GROUP.COM
3,"1, 4GROUP, INC",N999CB,EHM Status Report,N999CB@14GROUP.COM
4,1000218159 ONTARIO INC.,C-GLTG,Delinquent Data Alert,LANCE@TRUMBLE.CA
...,...,...,...,...
162333,ZULULAND PUBLISHING CC,ZS-FLJ,Delinquent Data Alert,KEVINCAMERON53@GMAIL.COM
162334,ZULULAND PUBLISHING CC,ZS-FLJ,EHM Status Report,KEVINCAMERON53@GMAIL.COM
162335,ZULULAND PUBLISHING CC,ZS-FLJ,Trend Alert,KEVINCAMERON53@GMAIL.COM
162336,ZVIKA GRINBERG,4X-BHZ,Delinquent Data Alert,ZVIKA@KISLEV.CO.IL


In [9]:
Alert_types.to_csv('Alert_types.csv', index=0)

In [5]:

selected_profile_IDs = pd.read_sql(f"""

select * from
(
select CUSTOMER,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,EMAIL from
(

select distinct
  e.NAME  AS CUSTOMER,E.ENTERPRISE_ID,rsi.ITEM_ID AS ASSET_ID,
  rr.DISPLAY_NAME AS ALERT_TYPE,
  rs.NAME as ALERT_NAME,
  v.REGNO,

  c.FIRSTNAME,
  c.LASTNAME,
  c.EMAIL

from SUBSCRIPTION.RPT_REPORT rr
join SUBSCRIPTION.RPT_SUB rs on rs.RPT_REPORT_ID = rr.RPT_REPORT_ID
join CSI.ENTERPRISE e on e.ENTERPRISE_ID = rs.ENTERPRISE_ID
join SUBSCRIPTION.RPT_SUB_ITEM rsi on rsi.RPT_SUB_ID = rs.RPT_SUB_ID
join SUBSCRIPTION.RPT_SUB_CONTACT rsc on rsc.RPT_SUB_ID = rs.RPT_SUB_ID
join CSI.CONTACT c on c.CONTACT_ID = rsc.CONTACT_PROFILE_ID and c.STATUS = '0'
JOIN CSI.LINKENTERPRISEREL ler ON ler.STATUS = '0' AND ler.CONTACT_ID = c.CONTACT_ID AND ler.PROFILE_ID = '0' and role_id <> 0
join SUBSCRIPTION.RPT_SUB_PROP rsp on rsp.RPT_SUB_ID = rs.RPT_SUB_ID and   rsp.IS_DELETED = 0
join SUBSCRIPTION.RPT_TRACKING rt on rt.RPT_SUB_ID = rs.RPT_SUB_ID and rt.IS_DELETED = 0
join TREND.V_ANALYZER_AC_DASHBOARD v on v.ACASSETID = rsi.ITEM_ID and ECTM_SUBSCRIPTION = 'Y' AND ENGPOS = 1

WHERE 1=1

AND ( e.name not like '%DEMO%' AND   e.name not like '%TEST%') 

  and rr.IS_DELETED = 0
  and rs.IS_DELETED = 0
  and rsi.IS_DELETED = 0
  and rsi.ITEM_TYPE = 'ASSET'
  and rsc.IS_DELETED = 0
    and c.email is not null
  and rs.CREATED_BY_USER not in ('EHM_AUTO_GEN')

order by
  rr.DISPLAY_NAME,
  v.REGNO,  c.FIRSTNAME,  c.LASTNAME

)
GROUP BY CUSTOMER,ALERT_TYPE,REGNO,EMAIL,ENTERPRISE_ID,ASSET_ID
ORDER BY 5 DESC
)l
--where cnt >1
ORDER BY CUSTOMER,REGNO,EMAIL,ALERT_TYPE

        
""", conn)


profile_IDs_total_data_2 = selected_profile_IDs.reset_index(drop=True)
profile_IDs_total_data_2

C:\Users\bezyb\AppData\Local\Temp\ipykernel_27088\3015192053.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  selected_profile_IDs = pd.read_sql(f"""


,CUSTOMER,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,EMAIL
0,0X25EED LLC,540102.DB1,42390,Delinquent Data Alert,N715JK,JAKE@GAMMA.AERO
1,0X25EED LLC,540102.DB1,42390,EHM Status Report,N715JK,JAKE@GAMMA.AERO
2,"1, 4GROUP, INC",483159.DB1,63567,Delinquent Data Alert,N999CB,N999CB@14GROUP.COM
3,"1, 4GROUP, INC",483159.DB1,63567,EHM Status Report,N999CB,N999CB@14GROUP.COM
4,1000218159 ONTARIO INC.,533033.DB1,326857,Delinquent Data Alert,C-GLTG,LANCE@TRUMBLE.CA
...,...,...,...,...,...,...
151622,ZULULAND PUBLISHING CC,366039.DB1,42670,Delinquent Data Alert,ZS-FLJ,KEVINCAMERON53@GMAIL.COM
151623,ZULULAND PUBLISHING CC,366039.DB1,42670,EHM Status Report,ZS-FLJ,KEVINCAMERON53@GMAIL.COM
151624,ZULULAND PUBLISHING CC,366039.DB1,42670,Trend Alert,ZS-FLJ,KEVINCAMERON53@GMAIL.COM
151625,ZVIKA GRINBERG,475476.DB1,115579,Delinquent Data Alert,4X-BHZ,ZVIKA@KISLEV.CO.IL


In [16]:
profile_IDs_total_data_2[profile_IDs_total_data_2['CUSTOMER']=='247 AVIATION LTD'].reset_index(drop=True)

,CUSTOMER,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,EMAIL
0,247 AVIATION LTD,538084.DB1,333907,Delinquent Data Alert,G-ISJE,BRIAN.SPURLE@247AVIATION.CO.UK
1,247 AVIATION LTD,538084.DB1,333907,EHM Status Report,G-ISJE,BRIAN.SPURLE@247AVIATION.CO.UK
2,247 AVIATION LTD,538084.DB1,333907,Fault Code Alert,G-ISJE,BRIAN.SPURLE@247AVIATION.CO.UK
3,247 AVIATION LTD,538084.DB1,333907,Trend Alert,G-ISJE,BRIAN.SPURLE@247AVIATION.CO.UK
4,247 AVIATION LTD,538084.DB1,333907,EHM Status Report,G-ISJE,RICHARD.JENKINS@247AVIATION.CO.UK
5,247 AVIATION LTD,538084.DB1,333907,Trend Alert,G-ISJE,RICHARD.JENKINS@247AVIATION.CO.UK
6,247 AVIATION LTD,538084.DB1,121741,Delinquent Data Alert,G-LUSO,BRIAN.SPURLE@247AVIATION.CO.UK
7,247 AVIATION LTD,538084.DB1,121741,EHM Status Report,G-LUSO,BRIAN.SPURLE@247AVIATION.CO.UK
8,247 AVIATION LTD,538084.DB1,121741,Trend Alert,G-LUSO,BRIAN.SPURLE@247AVIATION.CO.UK
9,247 AVIATION LTD,538084.DB1,121741,Delinquent Data Alert,G-LUSO,RICHARD.JENKINS@247AVIATION.CO.UK


In [4]:
selected_profile_IDs = pd.read_sql(f"""



select * from
(
select CUSTOMER, FIRSTNAME || ' ' || LASTNAME AS FULL_NAME, EMAIL, ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,SERVICENAME,ACMODEL,RECORDERS,PLOTGRAPHSERVICENAME,  count(*) as CNT  from
(

select distinct
  e.NAME  AS CUSTOMER,E.ENTERPRISE_ID,rsi.ITEM_ID AS ASSET_ID,
  rr.DISPLAY_NAME AS ALERT_TYPE,
  rs.NAME as ALERT_NAME,
  v.REGNO,

  c.FIRSTNAME,
  c.LASTNAME,
  c.EMAIL,

  v.SERVICENAME,
  v.ACMODEL,
  v.RECORDERS,
  v.PLOTGRAPHSERVICENAME


from SUBSCRIPTION.RPT_REPORT rr
join SUBSCRIPTION.RPT_SUB rs on rs.RPT_REPORT_ID = rr.RPT_REPORT_ID
join CSI.ENTERPRISE e on e.ENTERPRISE_ID = rs.ENTERPRISE_ID
join SUBSCRIPTION.RPT_SUB_ITEM rsi on rsi.RPT_SUB_ID = rs.RPT_SUB_ID
join SUBSCRIPTION.RPT_SUB_CONTACT rsc on rsc.RPT_SUB_ID = rs.RPT_SUB_ID
join CSI.CONTACT c on c.CONTACT_ID = rsc.CONTACT_PROFILE_ID and c.STATUS = '0'
JOIN CSI.LINKENTERPRISEREL ler ON ler.STATUS = '0' AND ler.CONTACT_ID = c.CONTACT_ID AND ler.PROFILE_ID = '0' and role_id <> 0
join SUBSCRIPTION.RPT_SUB_PROP rsp on rsp.RPT_SUB_ID = rs.RPT_SUB_ID and   rsp.IS_DELETED = 0
join SUBSCRIPTION.RPT_TRACKING rt on rt.RPT_SUB_ID = rs.RPT_SUB_ID and rt.IS_DELETED = 0
join TREND.V_ANALYZER_AC_DASHBOARD v on v.ACASSETID = rsi.ITEM_ID and ECTM_SUBSCRIPTION = 'Y' AND ENGPOS = 1

WHERE 1=1

AND ( e.name not like '%DEMO%' AND   e.name not like '%TEST%') 

  and rr.IS_DELETED = 0
  and rs.IS_DELETED = 0
  and rsi.IS_DELETED = 0
  and rsi.ITEM_TYPE = 'ASSET'
  and rsc.IS_DELETED = 0
    and c.email is not null
  and rs.CREATED_BY_USER not in ('EHM_AUTO_GEN')

order by
  rr.DISPLAY_NAME,
  v.REGNO , --FULL_NAME
  c.FIRSTNAME,  c.LASTNAME

)
GROUP BY CUSTOMER,ALERT_TYPE,REGNO,EMAIL,ENTERPRISE_ID,ASSET_ID,SERVICENAME, ACMODEL,RECORDERS, PLOTGRAPHSERVICENAME,FIRSTNAME, LASTNAME 
ORDER BY 5 DESC
)l
where cnt >1
ORDER BY CUSTOMER,REGNO,EMAIL,ALERT_TYPE


        
""", conn)


data_2 = selected_profile_IDs.reset_index(drop=True)
data_2

C:\Users\bezyb\AppData\Local\Temp\ipykernel_18336\1317362872.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  selected_profile_IDs = pd.read_sql(f"""


,CUSTOMER,FULL_NAME,EMAIL,ENTERPRISE_ID,ASSET_ID,ALERT_TYPE,REGNO,SERVICENAME,ACMODEL,RECORDERS,PLOTGRAPHSERVICENAME,CNT
0,247 AVIATION LTD,BRIAN SPURLE,BRIAN.SPURLE@247AVIATION.CO.UK,538084.DB1,121741,Delinquent Data Alert,G-LUSO,ESP Gold,PC-12/47E,None,Cruise Trend,2
1,AAR GOVERNMENT SERVICES,WILLIAM 'RICHARD' COX,COXW.CTR@GINL.STATE.GOV,468656.DB1,62788,Delinquent Data Alert,N560WK,RETAIL,DASH 8-300,None,Cruise Trend,2
2,AAR GOVERNMENT SERVICES,WILLIAM 'RICHARD' COX,COXW.CTR@GINL.STATE.GOV,468656.DB1,62788,EHM Status Report,N560WK,RETAIL,DASH 8-300,None,Cruise Trend,3
3,AAR GOVERNMENT SERVICES,RANDALL CREWS,CREWSR.CTR@GINL.STATE.GOV,468656.DB1,62788,Delinquent Data Alert,N560WK,RETAIL,DASH 8-300,None,Cruise Trend,2
4,AAR GOVERNMENT SERVICES,MARK HEITZMAN,HEITZMANM.CTR@GINL.STATE.GOV,468656.DB1,62788,Delinquent Data Alert,N560WK,RETAIL,DASH 8-300,None,Cruise Trend,2
...,...,...,...,...,...,...,...,...,...,...,...,...
8427,XOJET AVIATION LLC,JASON RAMOS,JASON.RAMOS@VISTAAMERICA.COM,91690.DB1,17309,EHM Status Report,N536XJ,FMP,BD-100-1A10,None,Cruise Trend,2
8428,YETI AIRLINES DOMESTIC PVT.LTD,SANTOSH NEPAL,S.NEPAL@YETIAIRLINES.COM,457147.DB1,109766,Delinquent Data Alert,9N-AMM,RETAIL,ATR72,FAST-400853,Cruise Trend,2
8429,ZANTAS AIR SERVICES LIMITED,MOHAMED ABYAT,M.ABYAT@YAHOO.COM,234037.DB1,27711,EHM Status Report,5H-FZA,Retail,208B CARAVAN - TBIP,FAST-450083,Cruise Trend,2
8430,ZANTAS AIR SERVICES LIMITED,MOHAMED ABYAT,M.ABYAT@YAHOO.COM,234037.DB1,27711,Trend Alert,5H-FZA,Retail,208B CARAVAN - TBIP,FAST-450083,Cruise Trend,2


In [5]:
list_of_alerts = data_2[['CUSTOMER', 'FULL_NAME', 'EMAIL', 'REGNO', 'ALERT_TYPE','SERVICENAME', 'ACMODEL',	'RECORDERS', 'PLOTGRAPHSERVICENAME', 'CNT']]
list_of_alerts.to_csv('list_of_alerts.csv', index=0)
list_of_alerts

,CUSTOMER,FULL_NAME,EMAIL,REGNO,ALERT_TYPE,SERVICENAME,ACMODEL,RECORDERS,PLOTGRAPHSERVICENAME,CNT
0,247 AVIATION LTD,BRIAN SPURLE,BRIAN.SPURLE@247AVIATION.CO.UK,G-LUSO,Delinquent Data Alert,ESP Gold,PC-12/47E,None,Cruise Trend,2
1,AAR GOVERNMENT SERVICES,WILLIAM 'RICHARD' COX,COXW.CTR@GINL.STATE.GOV,N560WK,Delinquent Data Alert,RETAIL,DASH 8-300,None,Cruise Trend,2
2,AAR GOVERNMENT SERVICES,WILLIAM 'RICHARD' COX,COXW.CTR@GINL.STATE.GOV,N560WK,EHM Status Report,RETAIL,DASH 8-300,None,Cruise Trend,3
3,AAR GOVERNMENT SERVICES,RANDALL CREWS,CREWSR.CTR@GINL.STATE.GOV,N560WK,Delinquent Data Alert,RETAIL,DASH 8-300,None,Cruise Trend,2
4,AAR GOVERNMENT SERVICES,MARK HEITZMAN,HEITZMANM.CTR@GINL.STATE.GOV,N560WK,Delinquent Data Alert,RETAIL,DASH 8-300,None,Cruise Trend,2
...,...,...,...,...,...,...,...,...,...,...
8427,XOJET AVIATION LLC,JASON RAMOS,JASON.RAMOS@VISTAAMERICA.COM,N536XJ,EHM Status Report,FMP,BD-100-1A10,None,Cruise Trend,2
8428,YETI AIRLINES DOMESTIC PVT.LTD,SANTOSH NEPAL,S.NEPAL@YETIAIRLINES.COM,9N-AMM,Delinquent Data Alert,RETAIL,ATR72,FAST-400853,Cruise Trend,2
8429,ZANTAS AIR SERVICES LIMITED,MOHAMED ABYAT,M.ABYAT@YAHOO.COM,5H-FZA,EHM Status Report,Retail,208B CARAVAN - TBIP,FAST-450083,Cruise Trend,2
8430,ZANTAS AIR SERVICES LIMITED,MOHAMED ABYAT,M.ABYAT@YAHOO.COM,5H-FZA,Trend Alert,Retail,208B CARAVAN - TBIP,FAST-450083,Cruise Trend,2


In [9]:
Alert_types['SERVICENAME'].unique()

array(['PPH-AVIALL', 'Retail', 'ESP Gold', 'ESP PA', 'RETAIL',
       'PPH-DUNCAN', 'MSP', 'ESP SILVER', 'ESP', 'ESP GOLD', 'ESP Silver',
       'ESP PLATINUM', 'FMP', 'ESP GOLD LITE', 'RETAIL - CJ4 24MONTH',
       'OEM-Dealer Package', 'ESP SILVER LITE', 'VECTOR', 'DETA',
       'FMP PHASE 3', 'FMP PHASE 2', 'RETAIL PHASE 3', 'FSP',
       'RETAIL PHASE 2', 'PW300 NETJETS', 'PW500 NETJETS'], dtype=object)

In [16]:
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'NHVM134'
DATABASE_NAME = 'Finance'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection=yes;
"""

conn = pyodbc.connect(connection_string)
print(conn)

In [17]:
Billable_Engines = pd.read_sql("SELECT * FROM [Billable_Engines_2022-03-31]  ", conn) 
Billable_Engines

C:\Users\bezyb\AppData\Local\Temp\ipykernel_31412\3755357345.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Billable_Engines = pd.read_sql("SELECT * FROM [Billable_Engines_2022-03-31]  ", conn)


,RunDt,SYSTEM,ProgramType,ProgramType2,IS_ACTIVE_CUSTOMER,CUSTOMER_NAME,AC_OEM_NAME,AC_MODEL_NAME,AC_PROFILE_ID,AC_ASSET_ID,...,AC_STATUS,ENG_POS,ENG_OEM_NAME,ENG_MODEL_MTX,ENG_MODEL_ALIAS_MTX,ENG_MODEL_TRIM,ENG_PROFILE_ID,ENG_ASSET_ID,ENG_SN,IS_DUPLICATE
0,2022-03-31 23:55:14.6700000,EHM,ESP GOLD,ESP,1,FLYEXCLUSIVE JETS,DASSAULT AVIATION S.A.,FALCON 2000LX,669746.DB1,58555.0,...,0,2,PWC,PW308C,PW308C,PW308C,781439.DB1,132001.0,CF0470,False
1,2022-03-31 23:55:14.6700000,EHM,PW500 NETJETS,FMP,1,"NETJETS, INC.",EMBRAER,EMB-505 - PHENOM 300,881392.DB1,30952.0,...,1,2,PWC,PW535E,PW535E,PW535E,894075.DB1,30924.0,DG0302,False
2,2022-03-31 23:55:14.6700000,EHM,FMP,FMP,1,"TRADEWIND AVIATION, LLC",PILATUS AIRCRAFT LTD,PC-12/47E,936794.DB1,23849.0,...,1,1,PWC,PT6A-67B/-67P,PT6A-67P,PT6A-67P,936795.DB1,23850.0,RY0526,False
3,2022-03-31 23:55:14.6700000,EHM,MSP,MSP,1,"HORUS AIRCRAFT HOLDINGS, INC",BOMBARDIER AVIATION,LEARJET 45,480736.DB1,3273.0,...,0,1,HONEYWELL,TFE731-20/-40/-60 (CIMMS),TFE731-20BR-1B,TFE731-20BR-1B,480750.DB1,10694.0,116848,False
4,2022-03-31 23:55:14.6700000,EHM,MSP,MSP,1,LEAF CAY AIR,ISRAEL AIRCRAFT INDUSTRIES LTD.,1124 WESTWIND,913253.DB1,3300.0,...,1,1,HONEYWELL,TFE731-3A-200G,TFE731-3D-1G,TFE731-3D-1G,914197.DB1,10741.0,77293,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33257,2022-03-31 00:00:00.0000000,TT,STANDALONE,STANDALONE,1,P&WC Test Accounts - HP PT-6 Training Account,None,PILATUS PC-12,999999997,NaN,...,0,1,PWC,None,None,PT6A-67B,48207,NaN,PR9998,False
33258,2022-03-31 00:00:00.0000000,TT,STANDALONE,STANDALONE,1,DEPARTMENT OF STATE - PAB,None,None,None,NaN,...,0,-1,PWC,None,None,PT6T-3B,None,NaN,TB0336,False
33259,2022-03-31 00:00:00.0000000,TT,STANDALONE,STANDALONE,1,GLOBAL VECTRA HELICORP LIMITED,None,BELL 412EP,597494089,NaN,...,0,1,PWC,None,None,PT6T-3D,None,NaN,TH1383,False
33260,2022-03-31 00:00:00.0000000,TT,STANDALONE,STANDALONE,1,GLOBAL VECTRA HELICORP LIMITED,None,BELL 412EP,738620285,NaN,...,0,1,PWC,None,None,PT6T-3DF,None,NaN,TH1365,True


In [28]:
Billable_Engines.columns

Index(['RunDt', 'SYSTEM', 'ProgramType', 'ProgramType2', 'IS_ACTIVE_CUSTOMER',
       'CUSTOMER_NAME', 'AC_OEM_NAME', 'AC_MODEL_NAME', 'AC_PROFILE_ID',
       'AC_ASSET_ID', 'REGNO', 'AC_SN', 'AC_MANAGED_BY', 'AC_STATUS',
       'ENG_POS', 'ENG_OEM_NAME', 'ENG_MODEL_MTX', 'ENG_MODEL_ALIAS_MTX',
       'ENG_MODEL_TRIM', 'ENG_PROFILE_ID', 'ENG_ASSET_ID', 'ENG_SN',
       'IS_DUPLICATE'],
      dtype='object')

In [23]:
Billable_Engines['ProgramType'].unique()

array(['ESP GOLD', 'PW500 NETJETS', 'FMP', 'MSP', 'RETAIL', 'ESP PA',
       'DETA', 'PPH-AVIALL', 'ESP PLATINUM', 'ESP', 'VECTOR',
       'ESP SILVER LITE', 'STANDALONE', 'FMP PHASE 3', 'PW300 NETJETS',
       'ESP SILVER', 'FMP PHASE 2', 'ESP GOLD LITE', 'OEM-DEALER PACKAGE',
       'JET-CARE', 'RETAIL - CJ4 24MONTH', 'ROLLS-ROYCE', 'FSP',
       'RETAIL PHASE 3', 'RETAIL - METCO OFFER', 'RETAIL PHASE 2',
       'ESP PAP', 'DAC-TREND', 'DAC-VAF', 'DAC-DETA', 'PWC-ESP',
       'DAC-CAMP'], dtype=object)

In [34]:
Billable_Engines [ Billable_Engines['ProgramType']=='DAC-VAF' ]['REGNO']

33065    F-HBCC
33066    F-GLNH
33068    F-GLNF
33069    VH-RUE
33070    F-GLNE
33071    F-HBCA
33073    F-GLNK
33074    F-GLNF
33075    F-GTKJ
33077    VH-RUE
33083    F-GLNE
33085    F-GOOB
33086    F-GOOB
33087    F-GRYL
33149    PR-MNS
33151    VT-ASU
33152    PR-MPK
33197    F-HAPE
33198    F-GOPE
33200    F-GLNH
33201    F-GLNK
33204    F-GTVC
33205    F-GUPE
33209    F-HBCB
33210    F-GLND
33211    F-HAPE
33212    F-HBCB
33213    F-GUPE
33214    F-GLND
33222    F-GRYL
33224    F-GUME
33225    F-GTVC
33226    F-GUME
33228    F-GTKJ
33229    F-GOPE
Name: REGNO, dtype: object

In [8]:
data_5['SERVICENAME'].unique()

array(['ESP PLATINUM', 'ESP GOLD', 'Retail', 'VECTOR', 'FMP', 'ESP',
       'ESP PA', 'ESP GOLD LITE', 'RETAIL', 'ESP Gold', 'ESP SILVER',
       'ESP Silver', 'ESP SILVER LITE', 'PW500 NETJETS', 'DETA',
       'PW300 NETJETS', 'FMP PHASE 3', 'FMP PHASE 2', 'RETAIL PHASE 3',
       'FSP', 'RETAIL PHASE 2', 'OEM-Dealer Package', 'MSP', 'PPH-DUNCAN',
       'PPH-AVIALL', 'RETAIL - CJ4 24MONTH', 'ROLLS-ROYCE',
       'Broker Package'], dtype=object)